# Model Prediction by Identifier
This notebook will run a regression on all of the data by the identifier(s) specified by the user. The regression is completed using SKLearn which utilizes test and training data to fit a learned model to the data. The model will then be used to complete a factor selection and forward/backward factor selection which can be used to fine-tune the model.

**How to Run This Code:** Run each segment of code and markdown in order using the 'Run' button above. Once you have run the last segment the code will execute and results will be outputted. It may take a few moments for the code to output results so please be patient.

### Import Libraries

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from time import time
from sklearn.feature_selection import SequentialFeatureSelector
import warnings
warnings.filterwarnings('ignore')

## Functions to be called later

The model is built and run in the section **"Calculating the Model"**.

**Calculating Model Validation:** This function calculates the r^2 anf RMSE of the model by utilizing the r2_score function with the test target data and predictions from the model, as well as the KFold function.

In [ ]:
def validations(preds, predictor_list, test, iden, train, model, X, y):
    
    # Calculates the r^2 score based on the test values and the predicted values
    r = r2_score(test["target"], preds)
    
    print("r2 of " + iden + " = " + str(r) + "\n")
    
    # Conducts the K_Fold test
    cv = KFold(n_splits=10, random_state=100, shuffle=True)

    scores = cross_val_score(model, train[predictor_list], train["target"],
                             scoring='neg_mean_absolute_error',
                             cv=cv, n_jobs=-1)
    
    print("root mean squared error (RMSE) = " + str(np.sqrt(np.mean(np.absolute(
        scores)))) + "\n")
    
    return r

**Selecting features with Sequential Feature Selection:** SFS is a greedy procedure where, at each iteration, we choose the best new feature to add to our selected features based a cross-validation score (starts with 0 features and choose the best single feature with the highest score). The procedure is repeated until we reach the desired number of selected features.

In [ ]:
# Simple helper function to go from list of True and False to the final selected values

def selecting(lst, features):
    selected = []
    for i in range(0,13):
        if lst[i] == True:
            selected.append(features[i])
    return selected

In [ ]:
def seq_selection(X, y, features, model, iden):

    # Forward Selection
    
    # starts the time
    tic_fwd = time()
    # runs the SequentialFeatureSelector on our ridge values previosuly calculated
    # and specifies that this is to be done in the forward direction
    sfs_forward = SequentialFeatureSelector(
        model, direction="forward"
    ).fit(X, y)
    # stops the time
    toc_fwd = time()

    #prints the results
    print(
        "Features of" + iden + " selected by forward sequential selection: ", selecting(sfs_forward.get_support(), features)
    )
    print(f"Done in {toc_fwd - tic_fwd:.3f}s")
    
    return selecting(sfs_forward.get_support(), features)

## Calculating the Model

**Set Path and Read Data:** Below I have specified the path to the given CSV containing the data. You do not need to change the path for it to work on this notebook, but if you would like to download the code, you may.

In [ ]:
path_to_file = "data/data.csv"
# Reads the information contained in the CSV
og_df = pd.read_csv(path_to_file)

**List all Unique Identifiers:** Lists all of the identifiers in the model which can be used to calculate the model

In [ ]:
unique_identifiers = og_df['identifier'].unique()
unique_identifiers

#### Please specify the identifier you would like to examine in the cell below
#### NOTE: User input required

In [ ]:
# list your identifiers here, i.e. iden = 'KM943MN5D7E3'
iden = 'KM943MN5D7E3'

**Get Predictors:** List the predictors that will be used in the model by filtering column names

In [ ]:
predictor_list = og_df.columns[2:-1]
predictor_list

**1) Arranging the dataset to factor the dates as the index and specify for the chosen identifier:** This will result in the index of the data being set as the dates. It will also filter the data by the chosen identifier

In [ ]:
ds = og_df.set_index('date')
ds = ds.loc[ds['identifier'] == iden]
ds.head(5)

**2) Partitioning the training and testing data:** 30% of the data is being selected as test data, while 70% is being used as training data. This selection is done at random, which may result in varied R^2 scores per run of the data. This is due to the fact that there is relatively little data per identifier.

In [ ]:
train_data, test_data = train_test_split(ds, test_size=0.3)

**3) Running Linear Model**: A linear regression on the training data is run using SkLearn LinearRegression().

In [ ]:
X, y = train_data[predictor_list], train_data["target"]

# Defines the model as a Linear Regression
model = LinearRegression()

# Fits the model using the predictors above and defined target training data
model.fit(X,y);

**4) Making predictions with all features and graphing the data vs predicted:** Predictions for the data are made using our training data and graphed against the test data to demonstrate accuracy.

In [ ]:
# Creates the predictors using the model.predict
preds = model.predict(test_data[predictor_list])

combined = np.vstack((test_data['target'], preds))

plt.plot(combined.T)
plt.grid()
plt.legend(("Target","Predictions"))
plt.title(iden);

**5) Calculating the R^2 and RMSE for the full model:** R^2 for the model is calculated using the function defined in the functions section of the code.

In [ ]:
r_2 = validations(preds, predictor_list, test_data, iden, train_data, LinearRegression(), X, y)

**6) Backward Selection:** Backward selection is conducted to determine the best predictors for the data which will be used to modify our model.

In [ ]:
new_predictors = seq_selection(X, y, predictor_list, model, iden)

**7) Optimizing feature selection using backward selection:** Using our new predictors from the backward selection, we run a linear regression using our training and testing data to formulate new predictions and a revised R^2 score.

In [ ]:
X2, y2 = train_data[new_predictors], train_data["target"]

# Defines the model as a Linear Regression
model = LinearRegression()

# Fits the model using the predictors above and defined target training data
model.fit(X2,y2)

new_preds = model.predict(test_data[new_predictors])

new_r_2 = validations(new_preds, new_predictors, test_data, iden, train_data, LinearRegression(), X2, y2)

**8) Graphing all of the models vs the test target data:** The new predictions are graphed against our previous predictions and the test target data.

In [ ]:
combined = np.vstack((test_data['target'], new_preds, preds))

plt.plot(combined.T)
plt.grid()
plt.legend(("Target","Reduced Model Predictions","Full Model Predictions"))
plt.title(iden);